In [1]:
get_ipython().magic(u'matplotlib inline')
import matplotlib.pyplot as plt
from urllib2 import Request, urlopen, URLError
import numpy as np
from skimage.util import view_as_windows as vaw
import tflearn
from tflearn.data_utils import to_categorical, pad_sequences
import tensorflow as tf
import os, sys
from scipy.stats import mode
import glob
from progressbar import ProgressBar

In [3]:
# if you have a single sequence you want to test,
# put it here between the quotes.
binding = 'MEFRMKVFKDVFTNDEVCSDSYVQQDPFEVPEFREIAFEVKSNKRIKGNEDYGIADNSEDAVEGMGADVEHVIDIVDSFQLTSTAFSKKEYSAYIKNYMQKVAKYLEEKKPDRVEIFKTKAQPFIKHILTNFDDFEFYMGESLDMEAGIIYSYYKGEEITPRFVYISDGLFEEKYLEHHHHHH'

In [4]:
class lstm_proteins:
    def __init__(self, kw, test_len, Train, 
                 num_examples, data=None, test_seq=None):
        '''Args:
                kw: uniprot keyword
                    
                test_len: amino acid sequence fragment length
                
                Train: whether to train a model or load one and test.
                
                num_examples: the number of proteins to pull from uniprot
                
                data: provided data not from uniprot. Should be in a 
                      dictionary structure with positive examples in 
                      data[1] and negative ones in data[0]
        '''
        
        assert((data is None) ^ (kw == 'User_Data')),'provide data or uniprot keyword, not both'
        self.test = test_seq
        self.test_len = test_len
        self.numxs = num_examples 
        self.kw = kw
        self.data = data
        self.X, self.Y = self.compile_data()
        
        if Train:
            self.train_network()     
            self.test_network()
        else:
            self.test_network()
        
        
    
    
    def get_uniprot_data(self):
        '''Goes to the uniprot website and searches for 
           data with the keyword given. Returns the data 
           found up to limit elements.'''

        kws = [self.kw, 'NOT+' + self.kw]
        Protein_data = {}
            
        for i in range(2):
            kw = kws[i]
            url1 = 'http://www.uniprot.org/uniprot/?query='
            url2 = '&columns=sequence&format=tab&limit='+str(self.numxs)
            query_complete = url1 + kw + url2
            request = Request(query_complete)
            response = urlopen(request)
            data = response.read()
            data = data.split('\n')
            data = data[1:]
            Protein_data[str(i)] = map(lambda x:x.lower(),data)
            
        x1 = Protein_data['0']
        x0 = Protein_data['1']    
        
        return x1, x0
    
    
    
    def letter2num(self, c):
        '''Assigns each letter in the amino acid code c
            a unique integer and chops the sequence into
            shorter sequences.'''
        
        if len(c[0]) > 1:
            X = np.zeros([0, self.test_len])
            pbar = ProgressBar()
            
            for seq in pbar(c):
                x = []
                
                for letter in seq:
                    x.append(max(ord(letter)-97, 0))
                    
                x = np.asarray(x)
                
                if x.size < self.test_len:
                    x = np.pad(x, self.test_len-x.size, 
                               'constant', constant_values=22.)
                    
                x = vaw(x, (self.test_len,))
                X = np.concatenate((X, x), 0)
                
                
        elif len(c[0]) == 1:
            X = []
            for i in c:
                X.append(ord(i)-97)
            X = np.asarray(X)
            
            if X.size < self.test_len:
                X = np.pad(X, self.test_len-X.size, 'constant',
                           constant_values=22.)
                
            X = vaw(X, (self.test_len))
            #X = X[::self.test_len - 10, :]
            
        return X

    

    
    def compile_data(self):
        '''Gets protein sequence data from get_uniprot_data
           and sends it to letter2num() to transform the 
           amino acid letters into numbers. Labels are then
           made for each example.'''

        if self.kw != 'User_Data':
            self.epochs = 6
            print('Getting data from uniprot...')
            self.x1, self.x0 = self.get_uniprot_data()
            
            self.x1 = self.letter2num(self.x1)
            self.x0 = self.letter2num(self.x0)
        
            len_diff = self.x1.shape[0] - self.x0.shape[0]

            if len_diff > 0:
                self.x1 = self.x1[:self.x1.shape[0]-len_diff, :]
            else:
                self.x0 = self.x0[:self.x0.shape[0] + len_diff, :]
    
            assert(self.x1.shape[0] == self.x0.shape[0]),'number of examples not equal'
        
        else:
            self.x1 = self.letter2num(self.data[1])
            self.x0 = self.letter2num(self.data[0])
            self.epochs = 50
        
        # compile data and make labels
        X = np.concatenate((self.x1, self.x0), 0)
        Y = np.zeros([self.x1.shape[0]+self.x0.shape[0], ])
        Y[:self.x1.shape[0]] = 1.0
        Y = to_categorical(Y, 2)
        
        return X, Y



    def LSTM(self, net, d_prob):
        '''The LSTM model used for training'''
    
        net = tflearn.embedding(net, input_dim=25, output_dim=25)
        net = tflearn.lstm(net, 300, dropout=d_prob, dynamic=False, return_seq=True)
        net = tflearn.lstm(net, 300, dropout=d_prob, dynamic=False, return_seq=True)
        net = tflearn.lstm(net, 300)
        net = tflearn.fully_connected(net, 150, activation='relu')
        net = tflearn.fully_connected(net, 2, activation='softmax')
        return net

    

    def num2letter(self, seq):
        '''Takes in a seq of amino acids composed of numbers 
           and transforms them into the respective letters.'''
    
        letters = []
    
        for i in xrange(len(seq)):
            letters.append(chr(int(seq[i]+97)))
        
        return letters
    
    
    
    def train_network(self):
        
        # send the input placeholder to the lstm 
        net = tflearn.input_data([None, self.test_len])
        net = self.LSTM(net, 0.5) # get the output of the network
        
        # define optimizer, learning rate, and objective function
        net = tflearn.regression(net, optimizer='adam',
                                 learning_rate=0.00001,
                                 loss='categorical_crossentropy')
    
        # instantiate the model
        self.model = tflearn.DNN(net, tensorboard_verbose=1)
        
        # start tensorboard to visualize training 
        os.system('tensorboard --logdir=/tmp/tflearn_logs/ &')
    
        # call the training method with its parameters
        self.model.fit(self.X, self.Y,  # data and labels
                  n_epoch=self.epochs,  # number of times to loop through dataset
                  validation_set=0.2, # proportion of data for validation
                  shuffle=True,  # shuffle the data randomly
                  show_metric=True,
                  batch_size=150,  # number to train on each iteration
                  snapshot_step=5000,  # how often to test validation set
                  run_id='ProteinNet_' + str(self.test_len) + '_' + '_LSTM_' + self.kw)
        
        self.model.save('lstm' + str(self.test_len) + self.kw)  # save the model and its parameters
        
        
        
        
    def test_network(self):
        fnames = glob.glob('*.index')
        
        try:
            self.model.predict((np.zeros([1, self.test_len])))
            self.test_seq = self.x1.tolist()
            
        except AttributeError:
            # send the input placeholder to the lstm 
            net = tflearn.input_data([None, self.test_len])
            net = self.LSTM(net, 1.0) # get the output of the network

            # instantiate the model
            self.model = tflearn.DNN(net, tensorboard_verbose=1)

            # load the saved model you want to test
            self.model.load('lstm' + str(self.test_len) + self.kw,
                            weights_only=True)
        
        best = np.zeros([0, self.test_len])
        best_out = []
        bestexs = []
        bestouts = []
        
        # go through the data again and find the sequence fragment
        # with the highest activation from each protein
        if self.test is not None:
            self.test_seq = map(lambda x:x.lower(), list(self.test))
            self.test_seq = np.asarray(self.test_seq)
            self.test_seq = self.test_seq[None, :].tolist()
        else: 
            if self.kw == 'User_Data':
                self.test_seq = self.data[1]
            else:
                self.test_seq, _ = self.get_uniprot_data()
            
        print('Finding consensus sequences in your data...')
        pbar = ProgressBar()
        
        for X1 in pbar(self.test_seq):
            X1 = self.letter2num(X1)
            x1out = np.asarray(self.model.predict(X1))
            highest_out = np.argmax(x1out[:, 1])
            best_out.append(x1out[highest_out, :])
            best_ex = X1[highest_out, :]
            
            if len(self.test_seq) == 1:
                print('This is the sequence found by the network.')
                print(self.num2letter(best_ex))
                sys.exit(0)
            
            best = np.concatenate((best, best_ex[None, :]), 0)
        best_out = np.asarray(best_out)
        
        print('These are the top 5 sequences found by the lstm network.')
        for i in range(5):
            bestexs.append(best[np.argmax(best_out), :])
            bestouts.append(np.amax(best_out))
            best = np.delete(best, np.argmax(best_out), 0)
            best_out = np.delete(best_out, np.argmax(best_out))

        print(bestexs)
        print(bestouts)

In [5]:
# open your own data like this
x1 = np.genfromtxt('BindProteins.txt', delimiter='\n', dtype='str')
x0 = np.genfromtxt('NoBindProteins.txt', delimiter='\n', dtype='str')
x1 = x1.tolist()
x0 = x0.tolist()

d = {}
d[1] = x1
d[0] = x0

In [6]:
# ask the user for a protein to find the consensus of 
kw = raw_input('''Enter the uniprot keyword for proteins to get
a consensus sequence from (e.g. homeobox, KW-0440, etc.). If you
have your own data, type None.
''')

if kw in ['None', 'none']:
    kw = 'User_Data'

Enter the uniprot keyword for proteins to get
a consensus sequence from (e.g. homeobox, KW-0440, etc.). If you
have your own data, type None.
homeobox


In [7]:
# prompt the user if they want to train or load a model
t = raw_input('''Type 'train' to train a model or 'test'
to test a model.
''')

if t in ['train', 'Train']:
    Train = True
else:
    Train = False

Type 'train' to train a model or 'test'
to test a model.
test


In [ ]:
# ask the user what sequence length they want to use
test_len = int(raw_input('''What sequence length do you want to use?
'''))

What sequence length do you want to use?
60


In [ ]:
network = lstm_proteins(kw,
                        test_len, 
                        Train, 
                        3500)

Getting data from uniprot...
